In [1]:
import pandas as pd
import glob
import re

In [2]:
df = pd.read_csv('G:/My Drive/GitHubProjects/MLS/data_files/scraped_raw/matches/feed/mls_match_feeds_no_dupes.csv')

In [3]:
df

,match_id,date,minute,title,comment,out_player,in_player
0,f6700cd1,Wednesday May 15 2024,NaN,END OF SECOND HALF,End of match.,NaN,NaN
1,f6700cd1,Wednesday May 15 2024,NaN,FULL TIME,"Second half ends, Chicago Fire FC 0, Charlotte...",NaN,NaN
2,f6700cd1,Wednesday May 15 2024,90'+6,Attempt Saved,Attempt saved. Carlos Terán's (Chicago Fire FC...,NaN,NaN
3,f6700cd1,Wednesday May 15 2024,90'+5,Attempt Saved,Attempt saved. Hugo Cuypers's (Chicago Fire FC...,NaN,NaN
4,f6700cd1,Wednesday May 15 2024,89',Attempt Missed,Attempt missed. Kerwin Vargas's (Charlotte FC)...,NaN,NaN
...,...,...,...,...,...,...,...
43705,914361ab,Sunday March 10 2024,15',Attempt Saved,Attempt blocked. Matti Peltola's (D.C. United)...,NaN,NaN
43706,914361ab,Sunday March 10 2024,11',Attempt Saved,Attempt saved. Jackson Hopkins's (D.C. United)...,NaN,NaN
43707,914361ab,Sunday March 10 2024,6',Attempt Saved,Attempt saved. Matt Miazga's (FC Cincinnati) w...,NaN,NaN
43708,914361ab,Sunday March 10 2024,3',Attempt Saved,Attempt saved. Mateusz Klich's (D.C. United) w...,NaN,NaN


In [4]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [5]:
month_map = {
        "january": "01", "february": "02", "march": "03", "april": "04",
        "may": "05", "june": "06", "july": "07", "august": "08",
        "september": "09", "october": "10", "november": "11", "december": "12",
        "jan": "01", "feb": "02", "mar": "03", "apr": "04",
        "jun": "06", "jul": "07", "aug": "08",
        "sep": "09", "oct": "10", "nov": "11", "dec": "12"
    }

In [6]:
### find float in date strings

float_dates = df[df["date"].apply(lambda x: isinstance(x, float))]
float_dates

,match_id,date,minute,title,comment,out_player,in_player
6741,f0b7242e,NaN,NaN,END OF SECOND HALF,End of match.,NaN,NaN
6742,f0b7242e,NaN,NaN,FULL TIME,"Second half ends, Columbus Crew 4, New York Ci...",NaN,NaN
6743,f0b7242e,NaN,90'+10,GOAL,Goal! 4:2. Jacen Russell-Rowe (Columbus Crew) ...,NaN,NaN
6744,f0b7242e,NaN,90'+9,Attempt Saved,Attempt saved. Diego Rossi's (Columbus Crew) w...,NaN,NaN
6745,f0b7242e,NaN,90'+6,NaN,NaN,Cucho Hernández,D. Chambost
...,...,...,...,...,...,...,...
30452,3c29cb63,NaN,4',Attempt Missed,Attempt missed. Diogo Gonçalves's (Real Salt L...,NaN,NaN
30453,3c29cb63,NaN,3',Attempt Saved,Attempt blocked. Amahl Pellegrino's (San Jose ...,NaN,NaN
30454,3c29cb63,NaN,2',Attempt Missed,Attempt missed. Cristian Arango's (Real Salt L...,NaN,NaN
30455,3c29cb63,NaN,1',Attempt Saved,Attempt blocked. Jackson Yueill's (San Jose Ea...,NaN,NaN


In [7]:
## fill match id f0b7242e with date "Saturday Aug 31, 2024"

df.loc[df['match_id'] == 'f0b7242e', 'date'] = 'Saturday Aug 31, 2024'

### fill match id 3c29cb63 with Saturday Aug 24, 2024

df.loc[df['match_id'] == '3c29cb63', 'date'] = 'Saturday Aug 24, 2024'

### fill match id bec23bed with Saturday Sep 20

df.loc[df['match_id'] == 'bec23bed', 'date'] = 'Saturday Sep 20'



In [8]:
## drop row # 16194 

df = df.drop(index=16194)

In [9]:
### find float in date strings

float_dates2 = df[df["date"].apply(lambda x: isinstance(x, float))]
float_dates2

,match_id,date,minute,title,comment,out_player,in_player


In [10]:
## remove day names from date strings

df['date'] = df['date'].apply(lambda x: ' '.join([word for word in x.split() if word not in days]))

In [11]:
df['date'] = df['date'].apply(
        lambda x: re.sub(
            r"([A-Za-z]+)",
            lambda m: month_map[m.group(1).lower()],
            x
        )
    )

df['date'] = df['date'].apply(lambda x: x + " 2025" if re.search(r"\d{4}$", x) is None else x)

### remove , from date strings

df['date'] = df['date'].apply(lambda x: x.replace(',', ''))

In [12]:
#### convert date column to datetime from mm dd yyyy format

df['date'] = pd.to_datetime(df['date'], format="%m %d %Y")

In [13]:
## if corner in comment, then title is corner

df['title'] = df.apply(lambda x: 'Corner' if pd.notna(x['comment']) and 'corner' in x['comment'].lower() else x['title'], axis=1)

df['title'] = df.apply(lambda x: 'Foul' if pd.notna(x['comment']) and 'foul' in x['comment'].lower() else x['title'], axis=1)

df['title'] = df.apply(lambda x: 'Offside' if pd.notna(x['comment']) and 'offside' in x['comment'].lower() else x['title'], axis=1)


In [14]:
clubs = [
    "Atlanta United",
    "Austin FC",
    "CF Montréal",
    "Charlotte FC",
    "Chicago Fire FC",
    "FC Cincinnati",
    "Colorado Rapids",
    "Columbus Crew",
    "D.C. United",
    "FC Dallas",
    "Houston Dynamo FC",
    "Sporting Kansas City",
    "LA Galaxy",
    "Los Angeles Football Club",
    "Inter Miami CF",
    "Minnesota United FC",
    "Minnesota United",
    "Nashville SC",
    "New England Revolution",
    "New York City Football Club",
    "New York City FC",
    "New York Red Bulls",
    "Orlando City",
    "Philadelphia Union",
    "Portland Timbers",
    "Real Salt Lake",
    "San Diego FC",
    "San Jose Earthquakes",
    "Seattle Sounders FC",
    "St. Louis CITY SC",
    "Toronto FC",
    "Vancouver Whitecaps FC"
]

team_map = {
    "Atlanta United": "ATL",
    "Austin FC": "ATX",
    "CF Montréal": "MTL",
    "Charlotte FC": "CLT",
    "Chicago Fire FC": "CHI",
    "FC Cincinnati": "CIN",
    "Colorado Rapids": "COL",
    "Columbus Crew": "CLB",
    "D.C. United": "DC",
    "FC Dallas": "DAL",
    "Houston Dynamo FC": "HOU",
    "Sporting Kansas City": "SKC",
    "LA Galaxy": "LA",
    "Los Angeles Football Club": "LAFC",
    "Inter Miami CF": "MIA",
    "Minnesota United": "MIN",
    "Minnesota United FC": "MIN",
    "Nashville SC": "NSH",
    "New England Revolution": "NE",
    "New York City Football Club": "NYC",
    "New York City FC": "NYC",
    "New York Red Bulls": "RBNY",
    "Orlando City": "ORL",
    "Philadelphia Union": "PHI",
    "Portland Timbers": "POR",
    "Real Salt Lake": "RSL",
    "San Diego FC": "SD",
    "San Jose Earthquakes": "SJ",
    "Seattle Sounders FC": "SEA",
    "St. Louis CITY SC": "STL",
    "Toronto FC": "TOR",
    "Vancouver Whitecaps FC": "VAN"
}




def assign_teams_from_fulltime(df):
    df = df.copy()

    # create output columns
    df["home_team"] = None
    df["away_team"] = None

    # Improved regex: allow apostrophes, accents, FC, SC, etc.
    pattern = re.compile(
        r"Second half ends,\s*([A-Za-zÀ-ÿ .'-]+?)\s+\d+,\s*([A-Za-zÀ-ÿ .'-]+?)\s+\d+",
        re.IGNORECASE
    )

    for mid, group in df.groupby("match_id"):

        ft = group[group["title"].str.contains("FULL TIME", case=False, na=False)]

        if len(ft) == 0:
            continue

        comment = ft.iloc[0]["comment"]

        match = pattern.search(comment)
        if not match:
            continue

        home_full = match.group(1).strip()
        away_full = match.group(2).strip()

        # 1. Exact match
        home_short = team_map.get(home_full)
        away_short = team_map.get(away_full)

        # 2. If exact match failed, try fuzzy match from clubs list
        if home_short is None:
            for club in clubs:
                if club.lower() in home_full.lower():
                    home_short = team_map.get(club)
                    break

        if away_short is None:
            for club in clubs:
                if club.lower() in away_full.lower():
                    away_short = team_map.get(club)
                    break

        # 3. Final fallback → None
        # (Later you can log missing clubs if needed)

        df.loc[df.match_id == mid, "home_team"] = home_short
        df.loc[df.match_id == mid, "away_team"] = away_short

    return df

In [15]:
df = assign_teams_from_fulltime(df)

In [16]:
df

,match_id,date,minute,title,comment,out_player,in_player,home_team,away_team
0,f6700cd1,2024-05-15,NaN,END OF SECOND HALF,End of match.,NaN,NaN,CHI,CLT
1,f6700cd1,2024-05-15,NaN,FULL TIME,"Second half ends, Chicago Fire FC 0, Charlotte...",NaN,NaN,CHI,CLT
2,f6700cd1,2024-05-15,90'+6,Attempt Saved,Attempt saved. Carlos Terán's (Chicago Fire FC...,NaN,NaN,CHI,CLT
3,f6700cd1,2024-05-15,90'+5,Attempt Saved,Attempt saved. Hugo Cuypers's (Chicago Fire FC...,NaN,NaN,CHI,CLT
4,f6700cd1,2024-05-15,89',Attempt Missed,Attempt missed. Kerwin Vargas's (Charlotte FC)...,NaN,NaN,CHI,CLT
...,...,...,...,...,...,...,...,...,...
43705,914361ab,2024-03-10,15',Attempt Saved,Attempt blocked. Matti Peltola's (D.C. United)...,NaN,NaN,CIN,DC
43706,914361ab,2024-03-10,11',Attempt Saved,Attempt saved. Jackson Hopkins's (D.C. United)...,NaN,NaN,CIN,DC
43707,914361ab,2024-03-10,6',Attempt Saved,Attempt saved. Matt Miazga's (FC Cincinnati) w...,NaN,NaN,CIN,DC
43708,914361ab,2024-03-10,3',Attempt Saved,Attempt saved. Mateusz Klich's (D.C. United) w...,NaN,NaN,CIN,DC


In [17]:
### check for na teams

df[df['home_team'].isna() | df['away_team'].isna()]

,match_id,date,minute,title,comment,out_player,in_player,home_team,away_team
16068,4f9ec18f,2024-04-06,22',Attempt Saved,Attempt saved. Amine Bassi's (Houston Dynamo F...,NaN,NaN,None,None
16069,4f9ec18f,2024-04-06,16',Attempt Saved,Attempt saved. Fabian Herbers's (Chicago Fire ...,NaN,NaN,None,None
16070,4f9ec18f,2024-04-06,NaN,KICK OFF,First half begins.,NaN,NaN,None,None
28213,9f82627f,2024-06-15,86',NaN,NaN,R. Navarro,D. Yapi,None,None
28214,9f82627f,2024-06-15,82',Attempt Missed,Attempt missed. Sebastián Driussi's (Austin FC...,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...
40201,0a6cd237,2024-06-08,16',Attempt Missed,Attempt missed. Tim Parker's (St. Louis CITY S...,NaN,NaN,None,None
40202,0a6cd237,2024-06-08,14',Attempt Missed,Attempt missed. Nathan Fogaça's (Portland Timb...,NaN,NaN,None,None
40203,0a6cd237,2024-06-08,10',Attempt Missed,João Klauss (St. Louis CITY SC) with header fr...,NaN,NaN,None,None
40204,0a6cd237,2024-06-08,3',Attempt Saved,Attempt saved. Jonathan Rodríguez's (Portland ...,NaN,NaN,None,None


In [18]:
### delete na team rows

df = df[~(df['home_team'].isna() | df['away_team'].isna())]

In [19]:
### delete row is 'lineups' is in comment

df = df[~df['comment'].str.contains('Lineups', na=False)]

df = df[~df['title'].str.contains('KICK OFF|HALF TIME|FULL TIME|END OF SECOND HALF', na=False)]

In [20]:
df

,match_id,date,minute,title,comment,out_player,in_player,home_team,away_team
2,f6700cd1,2024-05-15,90'+6,Attempt Saved,Attempt saved. Carlos Terán's (Chicago Fire FC...,NaN,NaN,CHI,CLT
3,f6700cd1,2024-05-15,90'+5,Attempt Saved,Attempt saved. Hugo Cuypers's (Chicago Fire FC...,NaN,NaN,CHI,CLT
4,f6700cd1,2024-05-15,89',Attempt Missed,Attempt missed. Kerwin Vargas's (Charlotte FC)...,NaN,NaN,CHI,CLT
5,f6700cd1,2024-05-15,87',NaN,NaN,João Pedro,J. Lindsey,CHI,CLT
6,f6700cd1,2024-05-15,84',Attempt Saved,Attempt blocked. Kerwin Vargas's (Charlotte FC...,NaN,NaN,CHI,CLT
...,...,...,...,...,...,...,...,...,...
43704,914361ab,2024-03-10,19',Attempt Saved,Attempt blocked. Cristian Dájome's (D.C. Unite...,NaN,NaN,CIN,DC
43705,914361ab,2024-03-10,15',Attempt Saved,Attempt blocked. Matti Peltola's (D.C. United)...,NaN,NaN,CIN,DC
43706,914361ab,2024-03-10,11',Attempt Saved,Attempt saved. Jackson Hopkins's (D.C. United)...,NaN,NaN,CIN,DC
43707,914361ab,2024-03-10,6',Attempt Saved,Attempt saved. Matt Miazga's (FC Cincinnati) w...,NaN,NaN,CIN,DC


In [21]:
## if minute is missing, delete

df = df[df['minute'].notna()]

In [22]:
### reverse minute sorting

df = df.iloc[::-1].reset_index(drop=True)


In [23]:
### if title is missing, fill with 'Substitution' 
df['title'] = df['title'].fillna('Substitution')

### move in_player and out_player to comment if title is Substitution
df['comment'] = df.apply(lambda x: f"Substitution: {x['out_player']} out, {x['in_player']} in" if x['title'] == 'Substitution' else x['comment'], axis=1)

In [24]:
### add feed_id per match 
df['feed_id'] = df.groupby('match_id').cumcount() + 1

In [25]:
df

,match_id,date,minute,title,comment,out_player,in_player,home_team,away_team,feed_id
0,914361ab,2024-03-10,3',Attempt Saved,Attempt saved. Mateusz Klich's (D.C. United) w...,NaN,NaN,CIN,DC,1
1,914361ab,2024-03-10,6',Attempt Saved,Attempt saved. Matt Miazga's (FC Cincinnati) w...,NaN,NaN,CIN,DC,2
2,914361ab,2024-03-10,11',Attempt Saved,Attempt saved. Jackson Hopkins's (D.C. United)...,NaN,NaN,CIN,DC,3
3,914361ab,2024-03-10,15',Attempt Saved,Attempt blocked. Matti Peltola's (D.C. United)...,NaN,NaN,CIN,DC,4
4,914361ab,2024-03-10,19',Attempt Saved,Attempt blocked. Cristian Dájome's (D.C. Unite...,NaN,NaN,CIN,DC,5
...,...,...,...,...,...,...,...,...,...,...
38652,f6700cd1,2024-05-15,84',Attempt Saved,Attempt blocked. Kerwin Vargas's (Charlotte FC...,NaN,NaN,CHI,CLT,30
38653,f6700cd1,2024-05-15,87',Substitution,"Substitution: João Pedro out, J. Lindsey in",João Pedro,J. Lindsey,CHI,CLT,31
38654,f6700cd1,2024-05-15,89',Attempt Missed,Attempt missed. Kerwin Vargas's (Charlotte FC)...,NaN,NaN,CHI,CLT,32
38655,f6700cd1,2024-05-15,90'+5,Attempt Saved,Attempt saved. Hugo Cuypers's (Chicago Fire FC...,NaN,NaN,CHI,CLT,33


In [26]:
## sort by date desc

df = df.sort_values(by=['date', 'match_id', 'feed_id'], ascending=[False, True, False]).reset_index(drop=False)

In [27]:
df

,index,match_id,date,minute,title,comment,out_player,in_player,home_team,away_team,feed_id
0,3242,04a4ef01,2025-10-18,90'+7,Attempt Missed,Attempt missed. Nicolás Romero's (Minnesota Un...,NaN,NaN,LA,MIN,36
1,3241,04a4ef01,2025-10-18,90'+7,Foul,Edwin Cerrillo (LA Galaxy) is shown the second...,NaN,NaN,LA,MIN,35
2,3240,04a4ef01,2025-10-18,90'+5,Corner,Goal! 2:1. Joaquín Pereyra (Minnesota United F...,NaN,NaN,LA,MIN,34
3,3239,04a4ef01,2025-10-18,89',Substitution,"Substitution: J. Paintsil out, T. Lepley in",J. Paintsil,T. Lepley,LA,MIN,33
4,3238,04a4ef01,2025-10-18,83',Attempt Saved,Attempt blocked. Joaquín Pereyra's (Minnesota ...,NaN,NaN,LA,MIN,32
...,...,...,...,...,...,...,...,...,...,...,...
38652,12781,1cfa7fdb,2024-02-21,23',Attempt Saved,Attempt saved. Diego Gómez's (Inter Miami CF) ...,NaN,NaN,MIA,RSL,5
38653,12780,1cfa7fdb,2024-02-21,21',Attempt Saved,Attempt blocked. Lionel Messi's (Inter Miami C...,NaN,NaN,MIA,RSL,4
38654,12779,1cfa7fdb,2024-02-21,20',Attempt Saved,Attempt saved. Luis Suárez's (Inter Miami CF) ...,NaN,NaN,MIA,RSL,3
38655,12778,1cfa7fdb,2024-02-21,15',Foul,Pablo Ruiz (Real Salt Lake) is shown the yello...,NaN,NaN,MIA,RSL,2


In [28]:
### remove in_player and out_player columns
df = df.drop(columns=['in_player', 'out_player'])

In [29]:
### reorder columns
df = df[['match_id', 'date', 'feed_id', 'minute', 'title', 'comment', 'home_team', 'away_team']]

In [30]:
df

,match_id,date,feed_id,minute,title,comment,home_team,away_team
0,04a4ef01,2025-10-18,36,90'+7,Attempt Missed,Attempt missed. Nicolás Romero's (Minnesota Un...,LA,MIN
1,04a4ef01,2025-10-18,35,90'+7,Foul,Edwin Cerrillo (LA Galaxy) is shown the second...,LA,MIN
2,04a4ef01,2025-10-18,34,90'+5,Corner,Goal! 2:1. Joaquín Pereyra (Minnesota United F...,LA,MIN
3,04a4ef01,2025-10-18,33,89',Substitution,"Substitution: J. Paintsil out, T. Lepley in",LA,MIN
4,04a4ef01,2025-10-18,32,83',Attempt Saved,Attempt blocked. Joaquín Pereyra's (Minnesota ...,LA,MIN
...,...,...,...,...,...,...,...,...
38652,1cfa7fdb,2024-02-21,5,23',Attempt Saved,Attempt saved. Diego Gómez's (Inter Miami CF) ...,MIA,RSL
38653,1cfa7fdb,2024-02-21,4,21',Attempt Saved,Attempt blocked. Lionel Messi's (Inter Miami C...,MIA,RSL
38654,1cfa7fdb,2024-02-21,3,20',Attempt Saved,Attempt saved. Luis Suárez's (Inter Miami CF) ...,MIA,RSL
38655,1cfa7fdb,2024-02-21,2,15',Foul,Pablo Ruiz (Real Salt Lake) is shown the yello...,MIA,RSL


In [31]:
### check for nan teams

df[df['home_team'].isna() | df['away_team'].isna()]

,match_id,date,feed_id,minute,title,comment,home_team,away_team


In [32]:
df

,match_id,date,feed_id,minute,title,comment,home_team,away_team
0,04a4ef01,2025-10-18,36,90'+7,Attempt Missed,Attempt missed. Nicolás Romero's (Minnesota Un...,LA,MIN
1,04a4ef01,2025-10-18,35,90'+7,Foul,Edwin Cerrillo (LA Galaxy) is shown the second...,LA,MIN
2,04a4ef01,2025-10-18,34,90'+5,Corner,Goal! 2:1. Joaquín Pereyra (Minnesota United F...,LA,MIN
3,04a4ef01,2025-10-18,33,89',Substitution,"Substitution: J. Paintsil out, T. Lepley in",LA,MIN
4,04a4ef01,2025-10-18,32,83',Attempt Saved,Attempt blocked. Joaquín Pereyra's (Minnesota ...,LA,MIN
...,...,...,...,...,...,...,...,...
38652,1cfa7fdb,2024-02-21,5,23',Attempt Saved,Attempt saved. Diego Gómez's (Inter Miami CF) ...,MIA,RSL
38653,1cfa7fdb,2024-02-21,4,21',Attempt Saved,Attempt blocked. Lionel Messi's (Inter Miami C...,MIA,RSL
38654,1cfa7fdb,2024-02-21,3,20',Attempt Saved,Attempt saved. Luis Suárez's (Inter Miami CF) ...,MIA,RSL
38655,1cfa7fdb,2024-02-21,2,15',Foul,Pablo Ruiz (Real Salt Lake) is shown the yello...,MIA,RSL


In [99]:
df.to_csv(
    "../../data_files/cleaned/matches/feed/cleaned_mls_match_feeds2.csv",
    index=False)

In [ ]:
def clean_feed(df):
    df = df.copy()
    
    df['title'] = df.apply(lambda x: 'Corner' if pd.notna(x['comment']) and 'corner' in x['comment'].lower() else x['title'], axis=1)

    df['title'] = df.apply(lambda x: 'Foul' if pd.notna(x['comment']) and 'foul' in x['comment'].lower() else x['title'], axis=1)

    df['title'] = df.apply(lambda x: 'Offside' if pd.notna(x['comment']) and 'offside' in x['comment'].lower() else x['title'], axis=1)   
    
    df = df[~df['comment'].str.contains('Lineups', na=False)]

    df = df[~df['title'].str.contains('KICK OFF|HALF TIME|FULL TIME|END OF SECOND HALF', na=False)]
    
    df = df[df['minute'].notna()]
    
    df = df.iloc[::-1].reset_index(drop=True)
    
    df['title'] = df['title'].fillna('Substitution')

    df['comment'] = df.apply(lambda x: f"Substitution: {x['out_player']} out, {x['in_player']} in" if x['title'] == 'Substitution' else x['comment'], axis=1)
    
    df['feed_id'] = df.groupby('match_id').cumcount() + 1
    
    df['date'] = df['date'].astype(str)
    
    df['date'] = df['date'].str.replace(r"^[A-Za-z]+ ", "", regex=True)

    month_map = {
        "january": "01", "february": "02", "march": "03", "april": "04",
        "may": "05", "june": "06", "july": "07", "august": "08",
        "september": "09", "october": "10", "november": "11", "december": "12"
    }

    df['date'] = df['date'].apply(
        lambda x: re.sub(
            r"[A-Za-z]+",
            lambda m: month_map.get(m.group(0).lower(), m.group(0)),
            x
        )
    )

    df['date'] = df['date'].apply(
        lambda x: x + " 2025" if not re.search(r"\b\d{4}\b", x) else x
    )
    
    df['date'] = pd.to_datetime(df['date'], format="%m %d %Y")
    
    df = df.drop(columns=['in_player', 'out_player'])

    df = df[['match_id', 'date', 'feed_id', 'minute', 'title', 'comment']]
    
    return df

In [ ]:
for file in glob.glob('G:/My Drive/GitHubProjects/MLS/data_files/scraped_raw/matches/feed/mls_match_feeds.csv'):
    df = pd.read_csv(file)
    cleaned_df = clean_feed(df)
    cleaned_df.to_csv(
        f"G:/My Drive/GitHubProjects/MLS/data_files/cleaned/matches/feed/cleaned_mls_match_feeds.csv",
        index=False)
